In [1]:
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import log_loss
import numpy as np
from fast_ml.model_development import train_valid_test_split
np.random.seed(0)
import optuna
import datetime 
from catboost import CatBoostClassifier
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from sklearn import preprocessing
import os
from catboost import CatBoostClassifier
import wget
from pathlib import Path
import shutil
import gzip

from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
def convert_to_one_hot(y):
    seen = False
    for i in range(len(y)):
        l = [0 for i in range(7)]
        l[y[i] - 1] = 1
        if not seen:
            y_o = np.array(l.copy())
            seen = True
        else:
            y_o = np.vstack([y_o, l.copy()])
    return y_o
def preprocess_data():
    
    train = pd.read_csv("train.csv")
    test = pd.read_csv("test.csv")
    y_train = train["Exited"].values
    y_test = test["Exited"].values
    x_train = train.drop(columns=["Exited"]).values
    x_test = test.drop(columns=["Exited"]).values
    
    return  x_train,y_train.squeeze(),x_test,y_test.squeeze()


In [3]:
X_train,y_train,X_test,y_test = preprocess_data()

In [4]:
y_train.shape

(8000,)

# Training

# XGB

In [5]:
n_estimators = 1000 if not os.getenv("CI", False) else 20

In [6]:
# A parameter grid for XGBoost
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }

In [7]:
from datetime import datetime
def timer(start_time=None):
    """ 
        Helps  to keep track of time elapsed while training.
        start time: if none then start time tracking
                    if not none tracks time from start time         
    """
    if not start_time:
        print(datetime.now())
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print("Time taken: %i hours %i minutes and %s seconds." % (thour, tmin, round(tsec, 2)))

In [8]:
def tuner(trial):    
    params = {        
        "n_estimators": trial.suggest_int("n_estimators", 1000, 4000, step=100),
        "max_depth":trial.suggest_int("max_depth", 2, 7),
        "min_child_weight": trial.suggest_int("min_child_weight", 7, 8),
        "learning_rate": trial.suggest_float("learning_rate", 1e-4, 1e-1, log=True), 
        "subsample": trial.suggest_float("subsample", 0.7, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.1, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-2, 10.),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-2, 10.),
        "gamma": trial.suggest_float("gamma", 0.2, 1.0, step=0.1),
    }    
    
    model = XGBClassifier(
        **params,
        n_jobs=-1,
        num_class=2,

        objective="multi:softmax",
        tree_method='gpu_hist', 
        gpu_id=0
    )
    model.fit(X_train, y_train,
            eval_set=[(X_test, y_test)],
            early_stopping_rounds=100,
            verbose=0)

    y_hat = model.predict_proba(X_test)
    
    return log_loss(y_test,y_hat)

start_time=timer(None)
study = optuna.create_study()
study.optimize(tuner, n_trials=100)
timer(start_time)

[I 2022-04-21 09:47:03,909] A new study created in memory with name: no-name-266afeb4-1cba-40c5-9113-0c06f316b19d
[W 2022-04-21 09:47:03,913] Trial 0 failed because of the following error: NameError("name 'XGBClassifier' is not defined")
Traceback (most recent call last):
  File "/home/neham/anaconda3/envs/seathru/lib/python3.9/site-packages/optuna/study/_optimize.py", line 213, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-8-e9cf8523bdbb>", line 14, in tuner
    model = XGBClassifier(
NameError: name 'XGBClassifier' is not defined


2022-04-21 09:47:03.909064


NameError: name 'XGBClassifier' is not defined

In [15]:
from xgboost import XGBClassifier

clf_xgb = XGBClassifier(max_depth=15,
    tree_method='gpu_hist',
    gpu_id=0,
    num_class=2,
    learning_rate=0.1,
    n_estimators=10000,
    verbosity=0,
    silent=None,
    objective="multi:softprob",
    booster='gbtree',
    n_jobs=-1,
    nthread=None,
    gamma=1.5,
    min_child_weight=3,
    max_delta_step=0,
    subsample=0.7,
    colsample_bytree=0.8,
    colsample_bylevel=1,
    colsample_bynode=1,
    reg_alpha=0,
    reg_lambda=1,
    scale_pos_weight=1,
    base_score=0.5,
    random_state=0,
    seed=None,)

clf_xgb.fit(X_test, y_test,
            eval_set=[(X_test, y_test)],
            early_stopping_rounds=40,
            verbose=10)

[0]	validation_0-mlogloss:0.64118
[10]	validation_0-mlogloss:0.39974
[20]	validation_0-mlogloss:0.33213
[30]	validation_0-mlogloss:0.30843
[40]	validation_0-mlogloss:0.29767
[50]	validation_0-mlogloss:0.29293
[60]	validation_0-mlogloss:0.28786
[70]	validation_0-mlogloss:0.28637
[80]	validation_0-mlogloss:0.28377
[90]	validation_0-mlogloss:0.28278
[100]	validation_0-mlogloss:0.28213
[110]	validation_0-mlogloss:0.28075
[120]	validation_0-mlogloss:0.27952
[130]	validation_0-mlogloss:0.27914
[140]	validation_0-mlogloss:0.27914
[150]	validation_0-mlogloss:0.27907
[160]	validation_0-mlogloss:0.27900
[170]	validation_0-mlogloss:0.27831
[180]	validation_0-mlogloss:0.27806
[190]	validation_0-mlogloss:0.27795
[200]	validation_0-mlogloss:0.27789
[210]	validation_0-mlogloss:0.27733
[220]	validation_0-mlogloss:0.27729
[230]	validation_0-mlogloss:0.27666
[240]	validation_0-mlogloss:0.27600
[250]	validation_0-mlogloss:0.27592
[260]	validation_0-mlogloss:0.27591
[270]	validation_0-mlogloss:0.27587
[28

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8,
              enable_categorical=False, gamma=1.5, gpu_id=0,
              importance_type=None, interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=15,
              min_child_weight=3, missing=nan, monotone_constraints='()',
              n_estimators=10000, n_jobs=-1, nthread=20, num_class=2,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              seed=0, silent=None, subsample=0.7, ...)

In [16]:
preds_test = np.array(clf_xgb.predict_proba(X_test))
test_acc = accuracy_score(y_pred=np.argmax(preds_test, axis=1), y_true=y_test)
print(test_acc)

0.8975


In [17]:
preds_test

array([[0.9237795 , 0.0762205 ],
       [0.98144376, 0.01855626],
       [0.48765254, 0.51234746],
       ...,
       [0.15286377, 0.8471362 ],
       [0.08211524, 0.9178848 ],
       [0.90298027, 0.09701973]], dtype=float32)

In [18]:
pd.DataFrame(preds_test).to_csv('xgboost_preds_shrutime.csv',index=False)


In [76]:
start_time = datetime.now()
X_train_scale = preprocessing.scale(X_train)
X_valid_scale = preprocessing.scale(X_test)
clf = LinearSVC(random_state=0, tol=1e-5)
clf.fit(X_train_scale, y_train)
print("SVC time execution(kernel='rbf' and C=1 ):",
     datetime.now() - start_time)


SVC time execution(kernel='rbf' and C=1 ): 0:00:00.435017


In [77]:
prob.shape

(2000, 2)

In [78]:
!pwd

/home/neham/apex/EE5180


In [79]:
p = np.array(clf.decision_function(X_valid_scale)) # decision is a voting function
prob = np.array([np.exp(p),1-np.exp(p)]).T
pd.DataFrame(prob).to_csv('svm_preds_shrutime.csv',index=False)


In [22]:

prediction = clf.predict(X_train_scale)
acc_score_train = accuracy_score(y_train, prediction)
prediction = clf.predict(X_valid_scale)
acc_score_valid = accuracy_score(y_test, prediction)
print("Train accuracy = {0}, Test accuracy = {1}".format(acc_score_train, acc_score_valid))

Train accuracy = 0.806875, Test accuracy = 0.8075


In [ ]:
pd.DataFrame(preds_test).to_csv('xgboost_preds_shrutime.csv')


In [71]:
cat_params = {
    'iterations': 5000,
    'learning_rate': 0.01,
    'od_wait': 1000,
    'depth': 10,
    'task_type' : 'GPU',
    'l2_leaf_reg': 4,
    'eval_metric': 'Accuracy',
    'devices' : '0',
    'verbose' : 1000
}
cat = CatBoostClassifier(**cat_params)
cat.fit(X_train, y_train, eval_set=(X_test, y_test))

0:	learn: 0.8628750	test: 0.8520000	best: 0.8520000 (0)	total: 43.7ms	remaining: 3m 38s
1000:	learn: 0.8835000	test: 0.8575000	best: 0.8580000 (428)	total: 50.3s	remaining: 3m 21s
2000:	learn: 0.8975000	test: 0.8570000	best: 0.8595000 (1341)	total: 2m 57s	remaining: 4m 25s
bestTest = 0.8595
bestIteration = 1341
Shrink model to first 1342 iterations.


In [80]:
preds_test = np.array(cat.predict_proba(X_test))
pd.DataFrame(preds_test).to_csv('cat_boost_preds_shrutime.csv',index=False)